Copy this file into the same folder as the data you want to analyse.

You can run a cell by hitting  `Shift + Enter` on the keyboard

A more extensive tutorial can be found [here](https://www.datacamp.com/community/tutorials/tutorial-jupyter-notebook#gs.kwHTUSQ) Skip to: "Getting Started With Jupyter Notebooks" if you are hazy.

In [ ]:
# Import and load some packages
%pylab notebook
import sfg2d
import os

# Create some folders
if not os.path.isdir("figures"): 
    os.mkdir("figures")
if not os.path.isdir("cache"):
    os.mkdir("cache")
    
# Monkeypatch some default values
sfg2d.core.VIS_WL = 812.7
sfg2d.core.PUMP_FREQ = 2400
#sfg2d.core.BASE_SPEC = Array_of_data
#sfg2d.core.NORM_SPEC = Array_of_data

# Reload cached results 
sample = sfg2d.SfgRecord("cache/")

# Backgroud

In [ ]:
# Fill in correct data path after the ./
# You can use the TAB key for autocompletion.
# Position your cursor after the ./ and hit the TAB key
bg0 = sfg2d.SfgRecord("./").make_avg().data # <- e.g. shot background scan
bg1 = sfg2d.SfgRecord("./").make_avg().data # <- e.g. long background scan

# Ir Profile

In [ ]:
q0 = sfg2d.SfgRecord("./") # Fill in path

q0 = q0.make_avg() # Calculates the frame wise average of a data set.
q0.base = bg0 # Choos correct baseline here. You can also just put a constan value.
q0.sub_base(inplace=True) # substracts baseline from data
q0._wavenumber = None # Reset Wavenumber so it gets recalculated using the above vis_wl
norm_spec = q0.data # <- This will be your dataset for normalization

In [ ]:
q0.plot(x_axis="wavenumber")
xlabel("Wavenumber 1/cm")
ylabel("Counts in {0}s".format(q0.metadata.get('exposure_time').seconds))
title("Quartz Normalization")
sfg2d.savefig("figures/norm") # Saves the figures as pixel and as vektor graphics.
q0.save("cache/quartz") # Saves intermediate result for easier reuse.

# Cross-Correlation

In [ ]:
cc0 = sfg2d.SfgRecord("./").make_avg() # Fill in path


cc0.base = bg0 # Use correct baseline
cc0.sub_base(inplace=True)
cc0._wavenumber = None

In [ ]:
fig, ax = subplots()
cc0.plot(y_pixel=slice(0,1), x_axis="wavenumber");
#xlim(4200, 5100) # Select some data range in x_axis coordinates
xlabel("Wavenumber in 1/cm")
ylabel("Counts in {0}s".format(cc0.metadata.get('exposure_time').seconds)
title("Pump-Probe IIV")
sfg2d.savefig("figures/cc_spectra")

fig, ax = subplots()
#x_slice = slice(363, 1277) # select some x_pixel before summing.
cc0.plot_trace(y_pixel_slice=slice(0, 1), x_pixel_slice=x_slice, as_mean=True)
xlabel("Time in fs")
ylabel("Mean Counts per Peak")
title("Cross-Correlation")
sfg2d.savefig("figures/cc_trace")

cc0.save("cache/pump_probe_iiv")

Fit of the Corss-Correlation Trace

In [ ]:
cc = cc0 # sfgrecord with cross correlation data
y_pixel = 1 # Where the cross correlation Spectrum is

xdata = cc.pp_delays
ydata = cc.get_trace_pp_delay(y_pixel_slice=[y_pixel]).squeeze()

ydata = ydata/ydata.max()
gm = sfg2d.models.GaussianModel(xdata, ydata, p0=[100, 0, 500])
gm.curve_fit()
gm.box_coords = (.1, .3)

fig, ax = subplots()
#cc.plot(x_axis="wavenumber")
gm.plot(show_fit_line=True, show_box=True)
legend()
xlabel("Time in fs")
ylabel("Intensity in a.u.")
title("Corss Correlation")
#sfg2d.savefig("figures/cc")

# Pump SFG

In [ ]:
# Pump-Sfg and Probe-SFG
pump = sfg2d.SfgRecord("").make_avg()
pump_sfg_spec_index = 0 # The y_pixel_index where the pump-sfg data is.
 
pump.base = bg0 # Set correct baseline
pump.norm = norm_spec
pump.sub_base(inplace=True)
pump._wavenumber = None
pump_spectra_slice = slice(pump_sfg_spec_index, pump_sfg_spec_index+1)
pump.save("cache/pump")

In [ ]:
fig, ax = subplots()
pump.plot(
    x_axis="wavenumber",
    y_pixel=pump_spectra_slice,
    
)
xlabel("Wavenumber in 1/cm")
ylabel("Counts in {0}s".format(pump.metadata.get('exposure_time').seconds))
#xlim(2300, 2600)
title("Pump SFG")
sfg2d.savefig("figures/pump")


fig, ax = subplots()
pump.plot_trace(y_pixel_slice=pump_spectra_slice, as_mean=True )
xlabel("Time in fs")
ylabel("Mean Counts")
title("Trace of Pump SFG Signal")
#xlim(-5000, 5000)
sfg2d.savefig("figures/pump_trace")

# Bleach

In [ ]:
g0 = sfg2d.SfgRecord("./").make_avg() # Fill in path

g0.base = bg0 # Use correct baseline
g0.norm = norm_spec
g0.sub_base(inplace=True)
g0._wavenumber = None
g0.pumped_index = 2
g0.unpumped_index = 1
g0.metadata["sample"] = "gold" # Sets a sample name
g0.metadata["pump_freq"] = 2300 # Sets a mean pump frequency
g0.save("cache/gold")

Pick a single pump-probe delay.

In [ ]:
fig, ax = subplots()
i = 1 # index of the example pump_probe delay 
g0.plot_bleach(attribute='bleach_rel', pp_delays=slice(i, i+1), x_axis='wavenumber');
xlabel("Wavenumber in 1/cm")
ylabel("Mean Relative Bleach")
title("Gold Relative Bleach {0}fs Delay".format(g0.pp_delays[i]))
sfg2d.savefig("figures/gold_bleach_rel_{0}fs".format(g0.pp_delays[i]))

Save all pump probe delays in a multipage pdf for easy browsing. The result of this is only saved to disk and not shown in the notebook.

In [ ]:
record = g0 # Use correct SfgRecord.
attribute = "bleach_rel"

@sfg2d.multipage_pdf # Within the decorator we hide the loop
def my_plot(index):
    record.plot_bleach(attribute=attribute, x_axis="wavenumber", pp_delays=[index]);
    title("{} @ {} fs".format(record.metadata.get("sample"), record.pp_delays[index]))
    # Usually the limits need some tweaking
    #xlim(2000, 3000) 
    #ylim(-0.015, 0.01)
    xlabel("wavenumber in 1/cm")

name = "figures/{}_{}_pump{}".format(
    record.metadata.get("sample"), attribute, record.metadata.get("pump_freq")
)
my_plot(name, range(record.number_of_pp_delays))

This is how you get a trace.

In [ ]:
fig, ax = subplots()
x_pixel_slice = slice(None, None) # Select some x_pixels before summing the bleach
g0.plot_trace(y_axis="get_trace_bleach", attr="bleach_rel", x_pixel_slice=x_pixel_slice);
xlabel("time in fs")
ylabel("Mean relative bleach")
title("Bleach on Gold")

# Combining Scans

In [ ]:
# Fill in paths to files
imp_data = (
    "./",  
    "./", 
)
data_list = []
for elm in imp_data:
    elm = sfg2d.SfgRecord(elm)
    ## Remove unneeded frames like so
    #if elm == "": # enter name of the file here
    #    sfgrecord = sfgrecord.keep_frames(slice(3)) # The second index is the frame slice
    data_list.append(elm)
record = sfg2d.concatenate_list_of_SfgRecords(data_list) # This cache the records into a single record

# Set some Parameters and Properties
record.metadata["pump_freq"] = 2500 # Mean freq of the pump
record.metadata["sample"] = "sample" # Name of the sample
record.base = bg1 # Baseline
record.norm = norm_spec # Set spectrum for normalization
record.pumped_index = 2 # Set index of pumped data
record.unpumped_index = 1 # Set index of unpumped data
record.sub_base(inplace=True) # Subtract baseline
record = record.make_static_corr() # Correct for static drifts in data
record = record.make_avg() # Make frame wise average
record.sub_base(inplace=True) # Make shure that baseline is still subtracted after corrections
record.normalize(inplace=True) # use ir profile to normalize data.
record._wavenumber = None # Reset Wavenumber for recalculation

# Apply some corrections
record = record.make_static_corr() # Correct for static drifts in data
record = record.make_avg() # Make frame wise average
record.sub_base(inplace=True) # Make shure that baseline is still subtracted after corrections
record.normalize(inplace=True) # use ir profile to normalize data.

# Save result
record.save("cache/{0}".format(record.metadata.get("sample")))

# Static Spectrum

In [ ]:
record = 

fig, ax = subplots()

record.plot(attribute="normalized", y_pixel=[1], x_axis="wavenumber")
ylim(0.005, 0.03)
xlim(2100, 2800)
xlabel("Wavenumber in 1/cm")
ylabel("Normalized SFG Intensity")
title("{0} Static Spectrum".format(record.metadata.get("sample")))
sfg2d.savefig("figures/{0}_normalized".format(record.metadata.get("sample")))

# 2D Spectrum from Scan

In [ ]:
record = 

# Data for axes
x = record.pp_delays
y = record.wavenumber[400:1200] # only use a pixel subset.
z = sfg2d.double_resample(record.bleach_rel.squeeze()[:, 400:1200], 30, axis=-1) # FFT based filters work better here
y_slice=(2300, 2500) # Touple with start and stop values for the summation
levels = linspace(-0.2, 0.05, 15) # Color levels and boundaries of the image

fig = plt.figure(figsize=(10.9, 7.5))
fig, ax, axl, axb = sfg2d.contour(
    x, y, z, 
    y_slice=y_slice,
    N=10, levels=levels, extend="both", show_colorbar=False, fig=fig) # vmin and vmax usually need some tweaking
axb.set_xlabel("Time in fs")
axl.set_ylabel("Wavenumber in 1/cm")
ax.set_xlim(-5000, 5000)
plt.tight_layout()

name = "figures/{}_bleach_pump{}_interval{}-{}_img".format(
    record.metadata.get("sample"), record.metadata.get("pump_freq"), *y_slice
)

sfg2d.savefig(name, dpi=80)

# Combine Traces

In [ ]:
record = 

fig, ax= subplots()
for peak_position in record.metadata["peak_positions"]:
    summation_slice = slice(*record.wavenumbers2index(peak_position, sort=True))
    record.plot_trace(
        y_axis="get_trace_bleach", 
        attr="bleach_rel", 
        x_pixel_slice=summation_slice,
        label="{} to {}".format(*peak_position)
    )
legend()
xlabel("Time in fs")
ylabel("Relaive Bleach")
title("{}".format(record.metadata["sample"]))

name = "figures/{}_bleach_rel_pump{}_trace".format(record.metadata.get("sample"), record.metadata.get("pump_freq"), *peak_position)
sfg2d.savefig(name)